In [90]:
import os
from functools import reduce
from glob import glob
from shutil import move
from xml.etree import ElementTree as et

import pandas as pd

In [91]:
xml_list = glob('./Imagens/*.xml')
xml_list = list(map(lambda x: x.replace('\\','/'),xml_list))

In [92]:
# step-2: read xml files
# from each xml file we need to extract
# filename, size(width, height), object(name, xmin, xmax, ymin, ymax)
def extract_text(filename):
    tree = et.parse(filename)
    root = tree.getroot()

    # extract filename
    image_name = root.find('filename').text
    # width and height of the image
    width = root.find('size').find('width').text
    height = root.find('size').find('height').text
    objs = root.findall('object')
    parser = []
    for obj in objs:
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = bndbox.find('xmin').text
        xmax = bndbox.find('xmax').text
        ymin = bndbox.find('ymin').text
        ymax = bndbox.find('ymax').text
        parser.append([image_name, width, height, name,xmin,xmax,ymin,ymax])
        
    return parser

In [93]:
parser_all = list(map(extract_text,xml_list))
data = reduce(lambda x, y : x+y,parser_all)

In [94]:
df = pd.DataFrame(data,columns = ['filename','width','height','name','xmin','xmax','ymin','ymax'])
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax
0,c_001.jpg,800,800,capacete,9,693,35,517
1,c_002.jpg,480,360,capacete,85,386,26,333
2,c_003.jpg,480,360,capacete,65,412,25,328
3,c_004.jpg,480,360,capacete,98,402,31,327
4,c_005.jpg,480,360,capacete,59,421,23,338


In [95]:
df['name'].value_counts()

name
luvas       807
capacete    539
mascara     471
oculos      261
Name: count, dtype: int64

In [96]:
cols = ['width','height','xmin','xmax','ymin','ymax']
df[cols] = df[cols].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2078 entries, 0 to 2077
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  2078 non-null   object
 1   width     2078 non-null   int32 
 2   height    2078 non-null   int32 
 3   name      2078 non-null   object
 4   xmin      2078 non-null   int32 
 5   xmax      2078 non-null   int32 
 6   ymin      2078 non-null   int32 
 7   ymax      2078 non-null   int32 
dtypes: int32(6), object(2)
memory usage: 81.3+ KB


In [97]:
df['center_x'] = ((df['xmax']+df['xmin'])/2)/df['width']
df['center_y'] = ((df['ymax']+df['ymin'])/2)/df['height']

df['w'] = (df['xmax']-df['xmin'])/df['width']

df['h'] = (df['ymax']-df['ymin'])/df['height']

In [98]:
img_df = pd.DataFrame(images,columns=['filename'])
img_train = tuple(img_df.sample(frac=0.8)['filename'])

In [99]:
img_test = tuple(img_df.query(f'filename not in {img_train}')['filename']) # take rest 20% images

In [100]:
train_df = df.query(f'filename in {img_train}')
test_df = df.query(f'filename in {img_test}')

In [101]:
def label_encoding(x):
    labels = {'capacete':0, 'oculos':1, 'luvas':2, 'mascara':3}
    return labels[x]

In [102]:
train_df['id'] = train_df['name'].apply(label_encoding)
test_df['id'] = test_df['name'].apply(label_encoding)

C:\Users\guiga\AppData\Local\Temp\ipykernel_18880\3116627190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['id'] = train_df['name'].apply(label_encoding)
C:\Users\guiga\AppData\Local\Temp\ipykernel_18880\3116627190.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['id'] = test_df['name'].apply(label_encoding)


In [103]:
cols = ['filename','id','center_x','center_y', 'w', 'h']
groupby_obj_train = train_df[cols].groupby('filename')
groupby_obj_test = test_df[cols].groupby('filename')

In [104]:
def save_data(filename, folder_path, group_obj):
    # move image
    src = os.path.join('Imagens',filename)
    dst = os.path.join(folder_path,filename)
    move(src,dst)
    
    text_filename = os.path.join(folder_path,
                                 os.path.splitext(filename)[0]+'.txt')
    group_obj.get_group(filename).set_index('filename').to_csv(text_filename,sep=' ',index=False,header=False)

In [105]:
filename_series = pd.Series(groupby_obj_train.groups.keys())
filename_series.apply(save_data,args=(train_folder,groupby_obj_train))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\guiga\AppData\Local\Programs\Python\Python39\lib\shutil.py", line 803, in move
    os.rename(src, real_dst)
FileNotFoundError: [WinError 2] O sistema não pode encontrar o arquivo especificado: 'Imagens\\L_001.jpg' -> 'Imagens/train\\L_001.jpg'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\guiga\Documents\Aplicativos\Object_Detection_Yolo\yolo_venv\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
  File "C:\Users\guiga\AppData\Local\Temp\ipykernel_18880\348021572.py", line 2, in <module>
    filename_series.apply(save_data,args=(train_folder,groupby_obj_train))
  File "c:\Users\guiga\Documents\Aplicativos\Object_Detection_Yolo\yolo_venv\lib\site-packages\pandas\core\series.py", line 4630, in apply
    return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
  File "c:\Users\guiga\Documents\Aplicativos\Object_Detection_Yolo\

In [106]:
filename_series_test = pd.Series(groupby_obj_test.groups.keys())
filename_series_test.apply(save_data,args=(test_folder,groupby_obj_test))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\guiga\AppData\Local\Programs\Python\Python39\lib\shutil.py", line 803, in move
    os.rename(src, real_dst)
FileNotFoundError: [WinError 2] O sistema não pode encontrar o arquivo especificado: 'Imagens\\L_006.jpg' -> 'Imagens/test\\L_006.jpg'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\guiga\Documents\Aplicativos\Object_Detection_Yolo\yolo_venv\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
  File "C:\Users\guiga\AppData\Local\Temp\ipykernel_18880\2966118263.py", line 2, in <module>
    filename_series_test.apply(save_data,args=(test_folder,groupby_obj_test))
  File "c:\Users\guiga\Documents\Aplicativos\Object_Detection_Yolo\yolo_venv\lib\site-packages\pandas\core\series.py", line 4630, in apply
    return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
  File "c:\Users\guiga\Documents\Aplicativos\Object_Detection_Yo